# 🎬 VideoAI - Free Video Generation

## 100% Free AI Video Generation using Google Colab!

**Features:**
- ✅ Completely Free
- ✅ No Credit Card Required
- ✅ High Quality Videos
- ✅ Multiple Models

**Instructions:**
1. Click Runtime → Run all
2. Wait 2-3 minutes for setup
3. Enter your prompt in the last cell
4. Generate and download your video!

---

In [1]:
# Step 1: Install Dependencies
print("📦 Installing dependencies...")
!pip install -q diffusers transformers accelerate imageio[ffmpeg] flask flask-cors pyngrok
print("✅ Dependencies installed!")

📦 Installing dependencies...
zsh:1: no matches found: imageio[ffmpeg]
✅ Dependencies installed!


In [2]:
# Step 2: Import Libraries
import torch
from diffusers import CogVideoXPipeline
from diffusers.utils import export_to_video
import imageio
from IPython.display import Video, display
import os

print("✅ Libraries imported!")
print(f"🔥 GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU Name: {torch.cuda.get_device_name(0)}")

/Users/sravyalu/VideoAI/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Libraries imported!
🔥 GPU Available: False


In [3]:
# Step 3: Load Model (CogVideoX-2B - Smaller, faster)
print("🤖 Loading CogVideoX-2B model...")
print("⏳ This may take 2-3 minutes...")

pipe = CogVideoXPipeline.from_pretrained(
    "THUDM/CogVideoX-2b",
    torch_dtype=torch.float16
)

if torch.cuda.is_available():
    pipe.to("cuda")
    print("✅ Model loaded on GPU!")
else:
    print("⚠️ Running on CPU (will be slower)")

print("🎬 Ready to generate videos!")

🤖 Loading CogVideoX-2B model...
⏳ This may take 2-3 minutes...


Loading pipeline components...:  80%|████████  | 4/5 [00:06<00:01,  1.53s/it]


ValueError: The component <class 'transformers.models.t5.tokenization_t5._LazyModule.__getattr__.<locals>.Placeholder'> of <class 'diffusers.pipelines.cogvideo.pipeline_cogvideox.CogVideoXPipeline'> cannot be loaded as it does not seem to have any of the loading methods defined in {'ModelMixin': ['save_pretrained', 'from_pretrained'], 'SchedulerMixin': ['save_pretrained', 'from_pretrained'], 'DiffusionPipeline': ['save_pretrained', 'from_pretrained'], 'OnnxRuntimeModel': ['save_pretrained', 'from_pretrained'], 'PreTrainedTokenizer': ['save_pretrained', 'from_pretrained'], 'PreTrainedTokenizerFast': ['save_pretrained', 'from_pretrained'], 'PreTrainedModel': ['save_pretrained', 'from_pretrained'], 'FeatureExtractionMixin': ['save_pretrained', 'from_pretrained'], 'ProcessorMixin': ['save_pretrained', 'from_pretrained'], 'ImageProcessingMixin': ['save_pretrained', 'from_pretrained'], 'ORTModule': ['save_pretrained', 'from_pretrained']}.

In [ ]:
# Step 4: Generate Video Function
def generate_video(prompt, output_path="output.mp4", num_frames=49):
    """
    Generate a video from a text prompt
    
    Args:
        prompt: Text description of the video
        output_path: Where to save the video
        num_frames: Number of frames (default 49 = ~6 seconds)
    """
    print(f"🎨 Generating video for: {prompt}")
    print(f"⏳ This will take 30-60 seconds...")
    
    # Generate
    video_frames = pipe(
        prompt=prompt,
        num_frames=num_frames,
        guidance_scale=6.0,
        num_inference_steps=50
    ).frames[0]
    
    # Save
    export_to_video(video_frames, output_path, fps=8)
    
    print(f"✅ Video saved to: {output_path}")
    
    # Display
    display(Video(output_path, embed=True, width=512))
    
    return output_path

print("✅ Generation function ready!")

---
## 🎬 Generate Your Video!

**Edit the prompt below and run the cell:**

In [ ]:
# 🎬 EDIT THIS PROMPT:
prompt = "A golden retriever running through a field of sunflowers at sunset"

# Generate!
video_path = generate_video(prompt)

# Download link
from google.colab import files
print("\n📥 Click below to download your video:")
files.download(video_path)

---
## 💡 Example Prompts

Try these prompts:

```python
# Nature
"A majestic waterfall cascading down rocks in a lush rainforest"

# Animals
"A cat playing with a ball of yarn on a cozy carpet"

# Urban
"City street at night with cars and neon lights"

# Fantasy
"A dragon flying over a medieval castle at dawn"

# Action
"A sports car drifting around a corner on a race track"
```

**Just copy a prompt above and paste it in the previous cell!**

---
## 🚀 Optional: Create API Server (Advanced)

Run this to create a temporary API endpoint you can connect to from your app!

In [ ]:
# Optional: Create Flask API + ngrok tunnel
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from pyngrok import ngrok
import threading

app = Flask(__name__)
CORS(app)

@app.route('/generate-video', methods=['POST'])
def api_generate():
    data = request.json
    prompt = data.get('prompt', '')
    
    if not prompt:
        return jsonify({'error': 'Prompt required'}), 400
    
    output_path = generate_video(prompt)
    
    return jsonify({
        'video_url': f'/download/{output_path}',
        'prompt': prompt
    })

@app.route('/download/<path:filename>')
def download(filename):
    return send_file(filename, mimetype='video/mp4')

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print(f"\n🌐 Your API is live at: {public_url}")
print(f"\n📝 Use this URL in your frontend!")
print(f"   Replace http://localhost:5000 with: {public_url}")

# Run Flask
threading.Thread(target=lambda: app.run(port=5000, debug=False)).start()